In [ ]:
from sklearn import datasets

digits = datasets.load_digits(10)

f, ax = plt.subplots(2, 5, sharex='col', sharey='row', figsize=(10, 8))

for i in range(2):
    for j in range(5):
        ax[i, j].imshow(
            digits.images[i * 5 + j, :], cmap='gray_r', interpolation='nearest')

In [ ]:
M = digits.images[:10].reshape((10, -1))